In [ ]:
import time
import pandas as pd
import pandapower as pp
import numpy as np
import matplotlib.pyplot as plt
import os
from pandapower.plotting.plotly import pf_res_plotly
from pandapower.plotting.plotly import simple_plotly

import Utils.LimitParadigm as LimitParadigm
import Utils.GA as GA
import Utils.utils as utils

# Import Network

In [ ]:
input_path = '../Data/Input'
output_path = '../Data/Output'

paradigm = LimitParadigm.LimitParadigm(input_path)

feeder_colors = ['#ff7f0e', '#9467bd']
feeder_colors_after = [utils.scale_lightness(c, 0.6) for c in feeder_colors]

meaningful_days = [15, 83, 162, 241, 324]
timesteps = paradigm.get_meaningful_days_timesteps(meaningful_days)

# Visualize Time-Series

In [ ]:
paradigm.plot_P()

In [ ]:
clips = ([-12,15], [-0.2,7.5]) #To be changed depending on your data
a = utils.plot_P_by_feeder(paradigm.B_init, paradigm, timesteps, feeder_colors, meaningful_days=meaningful_days, clips=clips)

# Initialize GA Algorithm

In [ ]:
#Set weigths for the different terms in the objective function
paradigm.scale_unbalance = 80 / (paradigm.number_timesteps * paradigm.number_customers) #weight for unbalance between the phases (Eq. 7 in the paper)
paradigm.scale_aggregate = 0 / (paradigm.number_timesteps * paradigm.number_customers) #weight to avoid worsening the situation at any step (not mentioned in the paper but may lead to a more robust solution)
paradigm.scale_changes = 4 / paradigm.number_customers #weight to reduce the number of changes (Eq 10)
paradigm.scale_distances = 1 / paradigm.number_customers #weight for considering the distance of the reconfiguration (Eq 11)

In [ ]:
ga_instance = GA.GA(paradigm)

In [ ]:
ga_instance.reconstruct = False
ga_instance.feeder = 0
ga_instance.initial_solution = paradigm.B_init_nobinary
# ga_instance.initial_solution = solution

if(ga_instance.feeder == 0):
    ga_instance.num_generations = 50
    ga_instance.population_size = 30
else:
    ga_instance.num_generations = 6
    ga_instance.population_size = 10
ga_instance.mutation_rate = 0.4
# ga_instance.initialize_run()

In [ ]:
initial_loss = paradigm.objective_function(paradigm.B_init, False)
print(f'Initial loss: {initial_loss}. \n Initial config: {paradigm.B_init_nobinary}')

In [ ]:
ga_instance.runGA()
if(ga_instance.reconstruct):
    solution = ga_instance.reconstruct_solution(ga_instance.best_solution[0])
else:
    solution = ga_instance.best_solution[0]

In [ ]:
#Some plots to see the different losses
plt.plot(np.array(paradigm.unbalance_loss) * paradigm.scale_unbalance, label='Unbalance')
plt.plot(np.array(paradigm.associated_loss) * paradigm.scale_aggregate, label='Associated_loss')
plt.plot(np.array(paradigm.changes_loss) * paradigm.scale_changes, label='Changes')
plt.plot(np.array(paradigm.loss_distance) * paradigm.scale_distances, label='Distance')
plt.legend()

In [ ]:
if(ga_instance.reconstruct):
    solution = ga_instance.reconstruct_solution(ga_instance.best_solution[0])
else:
    solution = ga_instance.best_solution[0]

In [ ]:
print(f'Initial loss: {initial_loss}.\nInitial config: {paradigm.B_init_nobinary}. Number customers: {len(paradigm.B_init_nobinary)}')
B_sol = paradigm.get_B_from_genetic(solution)
print(f'Solution loss: {paradigm.objective_function(B_sol, False)} ({paradigm.objective_function(B_sol)}). N. changes: {np.sum(B_sol * paradigm.B_init_opposite)}. \n Solution config: {solution}')
print([k for k in paradigm.B_init_nobinary])
print([k for k in solution])

In [ ]:
A_init, P = utils.plot_P_by_feeder(paradigm.B_init, paradigm, timesteps, feeder_colors, meaningful_days=meaningful_days, clips=clips)

In [ ]:
A_sol, P_sol = utils.plot_P_by_feeder(B_sol, paradigm, timesteps, feeder_colors, meaningful_days=meaningful_days, clips=clips)

In [ ]:
utils.plot_feeder_unbalance(paradigm, A_init, A_sol, feeder_colors, feeder_colors_after, meaningful_days=meaningful_days)

# Run PFs

In [ ]:
_, results = paradigm.run_simulations(paradigm.B_init, output_path)

In [ ]:
utils.plot_PF_results(paradigm, results)

In [ ]:
_, results_sol = paradigm.run_simulations(B_sol, output_path)

In [ ]:
utils.plot_PF_results(paradigm, results_sol)

In [ ]:
#Plot a graph to see the change positions
solution = paradigm.get_B_from_genetic(solution)
for i,changed in enumerate(np.sum(solution * paradigm.B_init_opposite, axis=1)):
    if(changed==1):
        c = paradigm.net.asymmetric_load.iloc[i] #it may give issues if the indexes are not the same as expected
        paradigm.net.bus.at[c['bus'], 'color'] = 'red'
simple_plotly(paradigm.net, bus_color=paradigm.net.bus['color'])